# LAB | Extractive Question Answering

This notebook demonstrates how Pinecone helps you build an extractive question-answering application. To build an extractive question-answering system, we need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A reader model to extract answers

We will use the SQuAD dataset, which consists of **questions** and **context** paragraphs containing question **answers**. We generate embeddings for the context passages using the retriever, index them in the vector database, and query with semantic search to retrieve the top k most relevant contexts containing potential answers to our question. We then use the reader model to extract the answers from the returned contexts.

Let's get started by installing the packages needed for notebook to run:

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('CarlosOPENAPI')
PINECONE_API_KEY= os.getenv('PINECONE_APY_KEY')

In [2]:
from google.colab import userdata
import os

# Retrieve the API keys
OPENAI_API_KEY = userdata.get('CarlosOPENAPI')
PINECONE_API_KEY = userdata.get('PINECONE_APY_KEY')

In [3]:
# Set as environment variables
os.environ['CarlosOPENAPI'] = OPENAI_API_KEY
os.environ['PINECONE_APY_KEY'] = PINECONE_API_KEY

print("OpenAI API key loaded and set as environment variable.")
print("Pinecone API key loaded and set as environment variable.")

OpenAI API key loaded and set as environment variable.
Pinecone API key loaded and set as environment variable.


# Install Dependencies

In [4]:
!pip install -qU datasets pinecone-client sentence-transformers torch

# Load Dataset

In [7]:
from google.colab import userdata
import os

HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE')
os.environ['HUGGINGFACE'] = HUGGINGFACE_TOKEN

Now let's load the SQUAD dataset from the HuggingFace Model Hub. We load the dataset into a pandas dataframe and filter the title, question, and context columns, and we drop any duplicate context passages.

In [8]:
from datasets import load_dataset

# load the squad dataset into a pandas dataframe
df = load_dataset("squad", split="train").to_pandas()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
df

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...
...,...,...,...,...,...
87594,5735d259012e2f140011a09d,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,"{'text': ['Oregon'], 'answer_start': [229]}"
87595,5735d259012e2f140011a09e,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,"{'text': ['Rangoon'], 'answer_start': [414]}"
87596,5735d259012e2f140011a09f,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,"{'text': ['Minsk'], 'answer_start': [476]}"
87597,5735d259012e2f140011a0a0,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,"{'text': ['1975'], 'answer_start': [199]}"


In [10]:
# select only title and context column
df1 = df[['title', 'context']].copy()



In [11]:
# drop rows containing duplicate context passages
df1 = df1.drop_duplicates(subset=['context'])

# Initialize Pinecone Index

The Pinecone index stores vector representations of our context passages which we can retrieve using another vector (query vector). We first need to initialize our connection to Pinecone to create our vector index. For this, we need a free [API key]("https://app.pinecone.io/"), and then we initialize the connection like so:

In [12]:
!pip install -qU langchain-pinecone pinecone-notebooks

In [15]:
from pinecone import Pinecone, ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

# connect to pinecone environment
pc = Pinecone(
    api_key = PINECONE_API_KEY,
    environment='us-east-1'  # find next to API key in console
)

Now we create a new index called "question-answering" — we can name the index anything we want. We specify the metric type as "cosine" and dimension as 384 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 384-dimension vectors.

In [16]:
index_name = "question-answering"

# check if the extractive-question-answering index exists
if index_name not in pc.list_indexes().names():
    # create the index if it does not exist
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of all-MiniLM-L6-v2
        metric='cosine',
        spec=spec
    )
# connect to extractive-question-answering index we created
index = pc.Index(index_name)

# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all context passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will generate embeddings in a way that the questions and context passages containing answers to our questions are nearby in the vector space. We can use cosine similarity to calculate the similarity between the query and context embeddings to find the context passages that contain potential answers to our question.

We will use a SentenceTransformer model named ``multi-qa-MiniLM-L6-cos-v1`` designed for semantic search and trained on 215M (question, answer) pairs from diverse sources as our retriever.

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

retriever = SentenceTransformer("all-MiniLM-L6-v2", device=device)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, context passage, etc.

In [ ]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df1), batch_size)):
    # find end of batch
    i_end = min(i + batch_size, len(df1))
    # extract batch
    batch = df1.iloc[i:i_end]
    # generate embeddings for batch
    # The retriever expects a list of strings
    emb = retriever.encode(batch['context'].tolist()).tolist()
    # get metadata
    meta = [{
        'text': row['context'],
        'title': row['title']
    } for idx, row in batch.iterrows()]
    # create unique IDs (using the original DataFrame index as ID)
    ids = [f"{idx}" for idx, row in batch.iterrows()]

    # add all to upsert list
    to_upsert = []
    for j in range(len(ids)):
        to_upsert.append({
            'id': ids[j],
            'values': emb[j],
            'metadata': meta[j]
        })
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

# Initialize Reader

We use the `deepset/electra-base-squad2` model from the HuggingFace model hub as our reader model. We load this model into a "question-answering" pipeline from HuggingFace transformers and feed it our questions and context passages individually. The model gives a prediction for each context we pass through the pipeline.

In [ ]:
from transformers import pipeline

model_name = 'deepset/electra-base-squad2'
# load the reader model into a question-answering pipeline
reader = pipeline(tokenizer=model_name, model=model_name, task='question-answering', device=device)
reader

Now all the components we need are ready. Let's write some helper functions to execute our queries. The `get_context` function retrieves the context embeddings containing answers to our question from the Pinecone index, and the `extract_answer` function extracts the answers from these context passages.

In [ ]:
# gets context passages from the pinecone index
#def get_context(question, top_k=3):
    # generate embeddings for the question
    #xq = retriever.encode(question).tolist()
    # search pinecone index for context passage with the answer
    #xc = index.query(vector=xq, top_k=top_k, include_metadata=True)
    # extract the context passage from pinecone search result
    #contexts = [match['metadata']['text'] for match in xc['matches']]
    # return the context passages as a list
    #return "\n".join(contexts)

In [ ]:
#from pprint import pprint

# extracts answer from the context passage
#def extract_answer(question, context):
    #results = []
    #for c in context:
        # feed the reader the question and contexts to extract answers
        #answer = reader(question=question, context=c)
        # add the context to answer dict for printing both together
        #answer["context"] = c
        #results.append(answer)
    # sort the result based on the score from reader model
    #sorted_result = pprint(sorted(results, key=lambda x: x['score'], reverse=True))
    #return sorted_result

In [ ]:
def get_context(question, top_k):
    # generate embeddings for the question
    xq = retriever.encode([question]).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(
        vector=xq,
        top_k=top_k,
        include_metadata=True # Important to get back the context text
    )
    # extract the context passage from pinecone search result
    c = [match.metadata['context'] for match in xc.matches]
    return c

In [ ]:
from pprint import pprint

# extracts answer from the context passage
def extract_answer(question, context):
    results = []
    for c in context:
        # feed the reader the question and contexts to extract answers
        answer = reader(question=question, context=c)
        # add the context to answer dict for printing both together
        answer["context"] = c
        results.append(answer)
    # sort the result based on the score from reader model
    sorted_result = pprint(sorted(results, key=lambda x: x['score'], reverse=True))
    return sorted_result

In [ ]:
question = "How much oil is Egypt producing in a day?"
context = get_context(question, top_k = 3)
context

As we can see, the retiever is working fine and gets us the context passage that contains the answer to our question. Now let's use the reader to extract the exact answer from the context passage.

In [ ]:
extract_answer(question, context)

The reader model predicted with 99% accuracy the correct answer *691,000 bbl/d* as seen from the context passage. Let's run few more queries.

In [ ]:
question = "What are the first names of the men that invented youtube?"
context = get_context(question, top_k=1)
extract_answer(question, context)

In [ ]:
question = "What is Albert Eistein famous for?"
context = get_context(question, top_k=1)
extract_answer(question, context)

In [ ]:
get_context("What is Albert Eistein famous for?", top_k=3)

Let's run another question. This time for top 3 context passages from the retriever.

In [ ]:
question = "What is the capital of France?"
context = get_context(question, top_k=3)


In [ ]:
question = "Who was the first person to step foot on the moon?"
context = get_context(question, top_k=3)
extract_answer(question, context)

In [ ]:
question = "Who was the first person to step foot on the Mars?"
context = get_context(question, top_k=3)
extract_answer(question, context)

In [ ]:
context

The result looks pretty good.

In [ ]:
#pc.delete_index(index_name)

### Add a few more questions. What did you observe?